# Setup

In [4]:
!pip install transformers transformers_interpret google-colab langdetect

     ---------------------------------------- 72.9/72.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 64.2/64.2 kB ? eta 0:00:00
     -------------------------------------- 104.5/104.5 kB 5.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of transformers-interpret to determine which version is compatible with other requirements. This could take a while.
  Using cached transformers_interpret-0.10.0-py3-none-any.whl (45 kB)
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 45.6/45.6 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 45.6/45.6 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 45.6/45.6 kB 2.4 MB/s eta 0:00:00
     -------------------------

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [843 lines of output]
  C:\Users\tiago\AppData\Local\Temp\pip-install-i6dzjgpb\pandas_c458349352fb410fae62002f565ab5ac\setup.py:50: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
    _CYTHON_INSTALLED = ver >= LooseVersion(min_cython_ver)
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\testing.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\_version.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\__init__.py -> build\lib.win-amd64-cpython-39\pandas
  creating build\lib.win-amd64-cpython-39\pandas\api
  copying pandas\api\__init__.py -> build\lib.win-amd64-cpython-39\pandas\api
  creating build\lib.wi

In [ ]:
from google.colab import drive

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from os import chdir
import pandas as pd
import nltk
from nltk.corpus import stopwords
from langdetect import detect
from string import punctuation
from transformers_interpret import SequenceClassificationExplainer

In [3]:
drive.mount('/content/drive')

In [10]:
REPO_DIR = '/content/drive/MyDrive/pantanal.dev/artificial-intelligence'
chdir(REPO_DIR)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/pantanal.dev/artificial-intelligence'

In [11]:
seed = 42
np.random.seed(seed)

In [22]:
train_df = pd.read_csv('../datasets/train_df.csv', sep='|')
val_df = pd.read_csv('../datasets/val_df.csv', sep='|')
test_df = pd.read_csv('../datasets/test_df.csv', sep='|')

In [13]:
# Carregar o tokenizer e o modelo BERT
model = BertForSequenceClassification.from_pretrained('Tuiaia/bert-base-multilingual-cased', output_attentions=True)
tokenizer = BertTokenizer.from_pretrained('Tuiaia/bert-base-multilingual-cased')

s:\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tiago\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# Classification result insights

In [14]:
nltk.download('stopwords')
nltk.download('punkt')

# Função para remover stopwords de um texto
def remove_stopwords(text: str, language: str = "auto") -> str:
    """
    Remove stopwords de um texto em português ou inglês.

    Args:
        text (str): O texto para remover as stopwords.
        language (str, optional): O idioma das stopwords. Por padrão, é utilizado "auto" para detectar o idioma automaticamente.

    Returns:
        str: O texto sem stopwords.

    Raises:
        ValueError: Se o idioma detectado automaticamente não for suportado pela aplicação.

    Exemplo de uso:
        >>> texto = "Este é um exemplo de texto em português que será processado para remoção de stopwords."
        >>> texto_sem_stopwords = remove_stopwords(texto, language="portuguese")
        >>> print(texto_sem_stopwords)
        "exemplo texto português processado remoção stopwords."
    """
    if language == 'auto':
        language = detect(text)
        if language == 'en':
            language = 'english'
        else:
            language = 'portuguese'

    words = nltk.word_tokenize(text)
    stopwords_list = set(stopwords.words(language))
    filtered_words = [word for word in words if word.lower() not in stopwords_list]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tiago\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tiago\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [15]:
def merge_subtokens(tokenizer, word_attributions: list[tuple]) -> list[tuple]:
    """
    Agrupa as pontuações de tokens divididos do BERT em um único token.
    
    Args:
        tokenizer: Tokenizer BERT para usar.
        word_attributions (List[tuple]): As atribuições de palavras do tokenizador.

    Returns:
        List[tuple]: Atribuições de palavras com tokens divididos mesclados.
    """
    merged_attributions = []
    merged_token = ''
    merged_value = 0.0

    for token, value in word_attributions:
        detokenized_token = tokenizer.convert_tokens_to_string([token]).strip()
        if detokenized_token:
            if detokenized_token in punctuation or detokenized_token in {'[CLS]', '[SEP]', '[UNK]'}:
                continue
            if token.startswith('##'):
                merged_token += token[2:]
                merged_value += value
            else:
                if merged_token:
                    merged_attributions.append((merged_token, merged_value))
                    merged_token = ''
                    merged_value = 0.0
                merged_token = detokenized_token
                merged_value = value
        else:
            merged_token += token.replace('##', '')
            merged_value += value

    if merged_token:
        merged_attributions.append((merged_token, merged_value))

    return merged_attributions

In [16]:
def format_attributions(word_attributions: list[tuple[str, float]]) -> list[tuple[str, str]]:
    """
    Formata as atribuições de palavras para uma representação mais intuitiva.

    Args:
        word_attributions: Uma lista de tuplas contendo as atribuições de palavras. Cada tupla contém uma palavra/token
            e um valor float representando sua importância.

    Returns:
        Uma lista de tuplas contendo as palavras formatadas e suas atribuições em formato de porcentagem, ordenadas por
        importância.
    """
    # Obter o valor total das atribuições
    total = sum(abs(score) for _, score in word_attributions)

    # Formatar cada atribuição como uma tupla (token, porcentagem)
    formatted_attributions = []
    for token, score in word_attributions:
        # Calcular a porcentagem e arredondar para duas casas decimais
        percentage = round((abs(score) / total) * 100, 2)
        formatted_attributions.append((token, percentage))

    # Ordenar as atribuições por porcentagem descendente
    formatted_attributions = sorted(formatted_attributions, key=lambda x: x[1], reverse=True)

    return formatted_attributions

In [17]:
cls_explainer = SequenceClassificationExplainer(model, tokenizer)

In [19]:
def classify_text(input_text):
    input_text = remove_stopwords(input_text)
    tokenized_input = tokenizer(input_text, truncation=True, max_length=100, return_tensors='pt')
    truncated_input_text = tokenizer.decode(tokenized_input['input_ids'][0])
    word_attributions = cls_explainer(truncated_input_text)
    word_attributions = merge_subtokens(tokenizer, word_attributions)
    word_attributions = sorted(word_attributions, key=lambda x: (-x[1], x[0]))
    word_attributions = format_attributions(word_attributions)

    return {
        'predicition_class_name': cls_explainer.predicted_class_name,
        'prediction_index': cls_explainer.predicted_class_index,
        'prediction_probatility': cls_explainer.pred_probs,
        'influential_words': word_attributions
    }

In [23]:
index = 8
text = test_df.loc[index, 'raw_text'], test_df.loc[index, 'label']
text

('O lucro operacional, excluindo itens não recorrentes, totalizou EUR 1,0 mn, abaixo dos EUR 1,6 mn.',
 0)

In [26]:
%timeit classify_text()

3 s ± 286 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
classify_text('Clientes sacaram R$ 850 bi de Credit Suisse e First Republic em crise do 1º tri')

{'predicition_class_name': 'Positive',
 'prediction_index': array(2, dtype=int64),
 'prediction_probatility': tensor(0.1679),
 'influential_words': [('sacaram', 24.14),
  ('crise', 20.89),
  ('1º', 16.02),
  ('First', 7.51),
  ('Suisse', 5.24),
  ('850', 5.05),
  ('Republic', 4.24),
  ('bi', 4.11),
  ('Credit', 3.47),
  ('tri', 3.22),
  ('Clientes', 3.17),
  ('R', 2.95)]}

In [40]:
import torch
import torch.nn.functional as F
def predict_sentiment(input_text, model):
    input_tokens = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=512)

    input_tokens.to('cpu')

    with torch.no_grad():
        output = model(**input_tokens)

    logits = output.logits
    probabilities = F.softmax(logits, dim=-1)
    predicted_class_idx = torch.argmax(probabilities, dim=1).item()

    return predicted_class_idx, probabilities

In [41]:
predict_sentiment('Clientes sacaram R$ 850 bi de Credit Suisse e First Republic em crise do 1º tri', model)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(0, tensor([[9.9952e-01, 9.5922e-05, 3.8229e-04]]))